In [8]:
import pandas as pd
from lifelines import CoxPHFitter
from src.helper_methods import *
from src.pipe_store import *
from src.constants import *
from lifelines import CoxPHFitter

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
RELEVANT_DESA_BAD = {
    '71TD', '144QL', '70DRA', '80TLR', '76ED', '70DA', '45EV', '67F', '76ET', '163RG', '67VG', '150AAH', '158T', '62QE', '45GV', '45KE', '71A', '66NH', '80TA', '114R', '44KM'
}

# Data Loading

### Total Cohort

In [10]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    # .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)

df = create_treatment_grups(df, [RELEVANT_DESA_BAD])

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.013283s
Step: integer_encoder | Shape: (4690, 12) | Computation Time: 0.000989s
Step: set_time_event_label | Shape: (4690, 14) | Computation Time: 0.001582s
Step: eng_immunological_features | Shape: (4690, 18) | Computation Time: 0.034805s
Step: censoring_deaths | Shape: (4690, 18) | Computation Time: 0.002384s


In [6]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    'DialysisYears',
    # 'No_DESA', 
    'Specific_DESA', 
    # 'Other_DESA'
]

In [11]:
formula = ' + '.join(confounders)

df_l = df
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['Specific_DESA'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 144


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 4690 total observations, 2293 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 4690
number of events observed = 2397
   partial log-likelihood = -18688.03
         time fit was run = 2023-01-24 12:14:09 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
CIPHour_DBD                0.17       1.19       0.02             0.12             0.22                 1.13                 1.24
CIPHour_DCD                0.19       1.21       0.02             0.15             0.24                 1.16                 1.27
DialysisYears              0.10       1.10       0.02             0.06             0.13                 1.06                 1.14
DonorAge_NOTR             -0.16       0.85       0.10            -0.35             0.03                 0.71                 1.03
DonorAge_NOTR_power2       0.38       1.46       0.09             0.20             0.57                 1.22                 1.76
IL2rMoAb                   0.19       1.21       0.06             0.08             0.30                 1.09                 1.35
RecipientAge_NOTR         -0.51       0.60       0.11            -0.71            -0.30                 0.49                 0.74
RecipientAge_NOTR_power2   0.78       2.18       0.10             0.58             0.98                 1.78                 2.65
Specific_DESA              0.66       1.93       0.10             0.46             0.85                 1.58                 2.35

                           cmp to     z      p   -log2(p)
covariate                                                
CIPHour_DBD                  0.00  7.09 <0.005      39.46
CIPHour_DCD                  0.00  8.69 <0.005      57.95
DialysisYears                0.00  4.85 <0.005      19.60
DonorAge_NOTR                0.00 -1.65   0.10       3.34
DonorAge_NOTR_power2         0.00  4.04 <0.005      14.21
IL2rMoAb                     0.00  3.44 <0.005      10.74
RecipientAge_NOTR            0.00 -4.82 <0.005      19.38
RecipientAge_NOTR_power2     0.00  7.65 <0.005      45.48
Specific_DESA                0.00  6.55 <0.005      33.97
---
Concordance = 0.64
Partial AIC = 37394.06
log-likelihood ratio test = 626.75 on 9 df
-log2(p) of ll-ratio test = 426.60

|Total Cohrot (4690)| #Tx  | Bad DESA (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |         144|      1.97  | 1.62-2.39

### Deceased Donors

In [39]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Deceased'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    # .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)

RELEVANT_DESA_BAD = {'71TD', '144QL', '70DRA', '80TLR', '76ED', '70DA', '45EV', '67F', '76ET', '163RG', '67VG', '150AAH', '158T', '62QE', '45GV', '45KE', '71A', '66NH', '80TA', '114R', '44KM'}
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.015928s
- selceted cohort --> Donor Type: Deceased, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (3235, 23) | Computation Time: 0.008305s
Step: integer_encoder | Shape: (3235, 12) | Computation Time: 0.00103s
Step: set_time_event_label | Shape: (3235, 14) | Computation Time: 0.001856s
Step: eng_immunological_features | Shape: (3235, 18) | Computation Time: 0.027639s
Step: censoring_deaths | Shape: (3235, 18) | Computation Time: 0.001765s


# Structural Causal Models

# Checking the Cox Proportional Hazard Assumptions

https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html?highlight=cox%20assumptions

The proportional hazard assumption is that all individuals have the same hazard function, but a unique scaling factor infront. So the shape of the hazard function is the same for all individuals, and only a scalar multiple changes per individual.

# Deceased Donors

In [40]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    'DialysisYears',
    # 'No_DESA', 
    'Specific_DESA', 
    # 'Other_DESA'
]

In [41]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# # # for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_late
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula)
len = df_l['Specific_DESA'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 75


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 2833 total observations, 1363 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 2833
number of events observed = 1470
   partial log-likelihood = -10700.25
         time fit was run = 2023-01-20 13:07:20 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
CIPHour_DBD               -0.02       0.98       0.04            -0.11             0.06                 0.90                 1.07
CIPHour_DCD                0.03       1.03       0.04            -0.06             0.11                 0.94                 1.12
DialysisYears              0.10       1.11       0.02             0.05             0.15                 1.05                 1.16
DonorAge_NOTR             -0.17       0.85       0.12            -0.41             0.08                 0.66                 1.08
DonorAge_NOTR_power2       0.39       1.48       0.12             0.15             0.63                 1.16                 1.88
IL2rMoAb                   0.15       1.16       0.07             0.01             0.29                 1.01                 1.33
RecipientAge_NOTR         -0.55       0.57       0.13            -0.82            -0.29                 0.44                 0.75
RecipientAge_NOTR_power2   0.87       2.39       0.13             0.62             1.13                 1.85                 3.09
Specific_DESA              0.56       1.75       0.14             0.28             0.84                 1.33                 2.31

                           cmp to     z      p   -log2(p)
covariate                                                
CIPHour_DBD                  0.00 -0.50   0.62       0.70
CIPHour_DCD                  0.00  0.60   0.55       0.86
DialysisYears                0.00  4.14 <0.005      14.78
DonorAge_NOTR                0.00 -1.33   0.18       2.46
DonorAge_NOTR_power2         0.00  3.20 <0.005       9.48
IL2rMoAb                     0.00  2.03   0.04       4.57
RecipientAge_NOTR            0.00 -4.16 <0.005      14.92
RecipientAge_NOTR_power2     0.00  6.70 <0.005      35.49
Specific_DESA                0.00  3.98 <0.005      13.82
---
Concordance = 0.63
Partial AIC = 21418.50
log-likelihood ratio test = 335.77 on 9 df
-log2(p) of ll-ratio test = 219.85

|Deceased donors (3235)| #Tx with Bad DESA | Bad DESA (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |         106|      2.01  | 1.62-2.51
| Early failures (< 1 y)| 31 |  1.60  | 1.10-2.32
| Late failures (>1 y)| 75  |    1.75|1.33-2.31

# Living Donor

In [48]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    # 'CIPHour_DBD', 'CIPHour_DCD', 
    # 'No_DESA', 
    'Specific_DESA', 
    'DialysisYears',
    # 'Other_DESA'
]

In [49]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'

donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])


Step: data_loading | Shape: (4690, 24) | Computation Time: 0.015168s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.007885s
Step: integer_encoder | Shape: (1455, 10) | Computation Time: 0.000901s
Step: set_time_event_label | Shape: (1455, 12) | Computation Time: 0.001721s
Step: eng_immunological_features | Shape: (1455, 16) | Computation Time: 0.014801s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.001819s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.002864s


In [89]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] >= 1]
df_early = df_total[df_total['T'] < 1]
df_class_I = df_total[df_total['Class_I'] == 1]
df_class_II = df_total[df_total['Class_II'] == 1]
df_class_I_II = df_total[df_total['Class_I_II'] == 1]

formula = ' + '.join(confounders)
# for df_l, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df_l = df_early
cph = CoxPHFitter()
cph.fit(df_l, duration_col='T', event_col='E', formula=formula, fit_options={'step_size':0.6})
len = df_l['Specific_DESA'].sum()
print(f' Bad DESA size in data set {len}')
cph.print_summary()

 Bad DESA size in data set 3


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 68 total observations, 2 right-censored observations>
             duration col = 'T'
                event col = 'E'
      baseline estimation = breslow
   number of observations = 68
number of events observed = 66
   partial log-likelihood = -206.51
         time fit was run = 2023-01-20 13:19:21 UTC

---
                           coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                        
DialysisYears              0.50       1.65       0.17             0.17             0.84                 1.18                 2.31
DonorAge_NOTR             -0.71       0.49       0.94            -2.55             1.14                 0.08                 3.12
DonorAge_NOTR_power2       0.83       2.30       0.90            -0.92             2.59                 0.40                13.35
IL2rMoAb                  -0.20       0.82       0.34            -0.87             0.47                 0.42                 1.61
RecipientAge_NOTR         -1.62       0.20       0.65            -2.89            -0.34                 0.06                 0.71
RecipientAge_NOTR_power2   1.39       4.01       0.62             0.17             2.61                 1.18                13.58
Specific_DESA              2.89      17.97       0.72             1.47             4.31                 4.35                74.34

                           cmp to     z      p   -log2(p)
covariate                                                
DialysisYears                0.00  2.94 <0.005       8.26
DonorAge_NOTR                0.00 -0.75   0.45       1.14
DonorAge_NOTR_power2         0.00  0.93   0.35       1.51
IL2rMoAb                     0.00 -0.58   0.57       0.82
RecipientAge_NOTR            0.00 -2.48   0.01       6.26
RecipientAge_NOTR_power2     0.00  2.23   0.03       5.28
Specific_DESA                0.00  3.99 <0.005      13.87
---
Concordance = 0.64
Partial AIC = 427.01
log-likelihood ratio test = 21.59 on 7 df
-log2(p) of ll-ratio test = 8.39

In [34]:
df_early.E.value_counts()

1    66
0     2
Name: E, dtype: int64

|Living donors (1455)| #Tx with Bad DESA | Bad DESA (HR) | 95% CI|
| ------------- | --------------| -----------|-----------|
| All           |        38 |       1.80      | 1.16-2.78	
| Early failures (< 1 y)| 3  |      17.97	  | 4.35-74.34
| Late failures (>1 y)| 35      |  1.79	 | 1.11-2.86

# Potential Outcomes

## Finding Inverse Probability Wieghts (IPW)

In [46]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df, confounders, treatments, scaler=None, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

PatsyError: Error evaluating factor: NameError: name 'CIPHour_DCD' is not defined
    No_DESA ~ RecipientAge_NOTR + RecipientAge_NOTR_power2 + DonorAge_NOTR + DonorAge_NOTR_power2 + IL2rMoAb + CIPHour_DBD + CIPHour_DCD
                                                                                                                             ^^^^^^^^^^^

In [ ]:
cph.check_assumptions(df_weight[cols], p_value_threshold=0.05, show_plots=True)

Proportional hazard assumption looks okay.


/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/kaplan_meier_fitter.py:317: StatisticalWarning: It looks like your weights are not integers, possibly propensity scores then?
  It's important to know that the naive variance estimates of the coefficients are biased. Instead use Monte Carlo to
  estimate the variances. See paper "Variance estimation when using inverse probability of treatment weighting (IPTW) with survival analysis"
  or "Adjusted Kaplan-Meier estimator and log-rank test with inverse probability of treatment weighting for survival data."
                  
  warnings.warn(
/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/fitters/mixins.py:108: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  for variable in self.params_.index & (colu

[]

In [ ]:
from lifelines.statistics import proportional_hazard_test
results = proportional_hazard_test(cph, df_weight[cols], time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/statistics.py:143: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return self.summary.to_latex()


<lifelines.StatisticalResult: proportional_hazard_test>
    time_transform = rank
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 9688.73 total observations, 5829.95 right-censored observations>
         test_name = proportional_hazard_test

---
               test_statistic    p  -log2(p)
Other_DESA               0.03 0.87      0.20
Specific_DESA            0.12 0.73      0.45

### Living Donor

In [ ]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type ='Living'
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        # 'TypeCadaveric_NOTR', 
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(features_from_antibody_epitopes)
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    # .pipe(feature_scaler, num_col, scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 15)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df['DESA_3more'] = df['#DESA'].apply(lambda x: 1 if x > 2 else 0)

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.01847s
- selceted cohort --> Donor Type: Living, Epitope Antibody Presence: All
Step: start_pipeline | Shape: (1455, 23) | Computation Time: 0.013016s
Step: integer_encoder | Shape: (1455, 12) | Computation Time: 0.000888s
Step: set_time_event_label | Shape: (1455, 16) | Computation Time: 0.004486s
Step: censoring_deaths | Shape: (1455, 16) | Computation Time: 0.002584s
Step: setting_prediction_horizon | Shape: (1455, 16) | Computation Time: 0.005771s


In [ ]:
df.sample()

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more
838,0,{},20,2.7,44,0.0,0.0,Living,0,1,0,0,0,0,0,15.0,0


In [ ]:
df_total = df.copy(deep=True)
df_late = df_total[df_total['T'] > 1]
df_early = df_total[df_total['T'] < 1]

# for df, title in [(df_total, 'Total Data Set'), (df_late, 'Late Failure'), (df_early, 'Early Failure')]:
df = df_total
confounders = [
    'RecipientAge_NOTR', 'RecipientAge_NOTR*RecipientAge_NOTR', 
    'DonorAge_NOTR', 'DonorAge_NOTR*DonorAge_NOTR', 'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
]

df_treat_group = create_treatment_grups(df, [RELEVANT_DESA_BAD])
# treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA', 'DESA_3more']
# df_weight = find_ipw(df_treat_group, confounders, treatments, verbose=False)
df_treat_group
# cph = CoxPHFitter()
# cols = [
#     'DonorAge_NOTR', 'RecipientAge_NOTR',
#     'DialysisYears', 'IL2rMoAb',
#     'CIPHour_DBD', 'CIPHour_DCD', 
#     'No_DESA',
#     'Specific_DESA',
#     'DESA_3more',
#     'E', 'T', 'w',
# ]
# cph.fit(df_weight[cols], weights_col='w', duration_col='T', event_col='E', robust=True)
# # print(title)
# cph.print_summary()


,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,TypeCadaveric_NOTR,Donor_Type,IL2rMoAb,...,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,E,T,DESA_3more,Specific_DESA,Other_DESA,Groups
0,0,{},44,2.3,54,9.000000,0.000000,0,Deceased,0,...,0,0,0,0,0,12.191781,0,0,0,1
1,0,{},54,2.1,32,21.750000,0.000000,0,Deceased,0,...,0,0,0,0,0,15.000000,0,0,0,1
2,0,{},44,4.9,27,36.000000,0.000000,0,Deceased,0,...,0,0,0,0,1,6.956164,0,0,0,1
3,0,{},23,0.0,40,30.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,4.509589,0,0,0,1
4,0,{},55,5.4,38,0.000000,17.216667,1,Deceased,0,...,0,0,0,0,1,0.010959,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},51,1.5,43,29.000000,0.000000,0,Deceased,1,...,0,0,0,0,0,12.126027,0,0,0,1
4686,0,{},54,1.7,37,20.700000,0.000000,0,Deceased,1,...,0,0,0,0,1,10.879452,0,0,0,1
4687,0,{},21,3.5,46,17.183333,0.000000,0,Deceased,1,...,0,0,0,0,0,15.000000,0,0,0,1
4688,0,{},42,2.5,23,0.000000,32.183333,1,Deceased,1,...,0,0,0,0,1,11.816438,0,0,0,1


# Total Data

In [ ]:
path = '~/Repos/STRIDE/STRIDE-Analytics/data/20210614-mismatch_ep_db-extended.pickle'
donor_type = None
status = 'All'
num_col = ['DonorAge_NOTR', 'DialysisYears', 'RecipientAge_NOTR', 'CIPHour_DBD', 'CIPHour_DCD']

df = (
    data_loading(path)
    # .pipe(start_pipeline, status, donor_type)
    .pipe(keeping_features,
        'Failure',
        'Survival[Y]',
        'EpvsHLA_Donor',
        '#DESA',
        'DESA', 
        'DonorAge_NOTR',
        'DialysisYears',
        'RecipientAge_NOTR',
        'CIPHour_DBD', 
        'CIPHour_DCD' ,
        'Donor_Type',
        'IL2rMoAb_T0'
    )
    .pipe(integer_encoder, 'IL2rMoAb_T0')
    .pipe(polynomial_power2, 'DonorAge_NOTR', 'RecipientAge_NOTR')
    .pipe(set_time_event_label, E='Failure', T='Survival[Y]')
    .pipe(eng_immunological_features, antibody_epitope=True)
    .pipe(feature_scaler, num_col + ['DonorAge_NOTR_power2', 'RecipientAge_NOTR_power2'], scaler='standard')
    .pipe(censoring_deaths)
    .pipe(setting_prediction_horizon, 10)
    .rename(columns={'IL2rMoAb_T0':'IL2rMoAb'})
)
df = create_treatment_grups(df, [RELEVANT_DESA_BAD])

Step: data_loading | Shape: (4690, 24) | Computation Time: 0.013059s
Step: integer_encoder | Shape: (4690, 12) | Computation Time: 0.001571s
Step: set_time_event_label | Shape: (4690, 14) | Computation Time: 0.001867s
Step: eng_immunological_features | Shape: (4690, 19) | Computation Time: 0.038885s
Step: censoring_deaths | Shape: (4690, 19) | Computation Time: 0.001222s
Step: setting_prediction_horizon | Shape: (4690, 19) | Computation Time: 0.004254s


In [ ]:
df

,#DESA,DESA,DonorAge_NOTR,DialysisYears,RecipientAge_NOTR,CIPHour_DBD,CIPHour_DCD,Donor_Type,IL2rMoAb,DonorAge_NOTR_power2,...,T,No_DESA,Relevant_DESA_Bad,Relevant_DESA_Good,Class_I,Class_II,Class_I_II,Specific_DESA,Other_DESA,Groups
0,0,{},-0.019920,-0.183046,0.596332,-0.190025,-0.433789,Deceased,0,-0.200724,...,10.000000,1,0,0,0,0,0,0,0,1
1,0,{},0.645513,-0.263943,-0.936160,0.857348,-0.433789,Deceased,0,0.579189,...,10.000000,1,0,0,0,0,0,0,0,1
2,0,{},-0.019920,0.868621,-1.284454,2.027942,-0.433789,Deceased,0,-0.200724,...,6.956164,1,0,0,0,0,0,0,0,1
3,0,{},-1.417332,-1.113366,-0.378890,1.535061,-0.433789,Deceased,1,-1.320455,...,4.509589,1,0,0,0,0,0,0,0,1
4,0,{},0.712057,1.070865,-0.518208,-0.929348,1.549608,Deceased,0,0.665934,...,0.010959,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4685,0,{},0.445883,-0.506635,-0.169914,1.452914,-0.433789,Deceased,1,0.328503,...,10.000000,1,0,0,0,0,0,0,0,1
4686,0,{},0.645513,-0.425738,-0.587867,0.771094,-0.433789,Deceased,1,0.579189,...,10.000000,1,0,0,0,0,0,0,0,1
4687,0,{},-1.550418,0.302339,0.039062,0.482210,-0.433789,Deceased,1,-1.390488,...,10.000000,1,0,0,0,0,0,0,0,1
4688,0,{},-0.153007,-0.102148,-1.563089,-0.929348,3.273799,Deceased,1,-0.337606,...,10.000000,1,0,0,0,0,0,0,0,1


In [ ]:
confounders = [
    'RecipientAge_NOTR', 
    'RecipientAge_NOTR_power2',
    'DonorAge_NOTR',
    'DonorAge_NOTR_power2',
    'IL2rMoAb', 
    'CIPHour_DBD', 'CIPHour_DCD', 
    'Donor_Type', 
    'DialysisYears',

]
treatments = ['No_DESA', 'Other_DESA', 'Specific_DESA']
df_weight = find_ipw(df, confounders, treatments, scaler=None, verbose=False)

cph = CoxPHFitter()
cols = [
    # 'No_DESA',
    'Specific_DESA',
    'Other_DESA',
    'E', 'T', 
    'w'
]
cph.fit(
    df_weight[cols], 
    weights_col='w',
    duration_col='T', event_col='E', robust=True)
print(cph.print_summary())

/Users/Danial/.local/share/virtualenvs/desa_paper-IhoHddMh/lib/python3.8/site-packages/lifelines/utils/printer.py:62: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  return summary_df[columns].to_latex(float_format="%." + str(self.decimals) + "f")


<lifelines.CoxPHFitter: fitted with 14035.4 total observations, 9843.98 right-censored observations>
             duration col = 'T'
                event col = 'E'
              weights col = 'w'
          robust variance = True
      baseline estimation = breslow
   number of observations = 14035.4
number of events observed = 4191.42
   partial log-likelihood = -38510.38
         time fit was run = 2022-06-28 12:17:43 UTC

---
                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                             
Specific_DESA   1.03       2.80       0.12             0.80             1.26                 2.22                 3.53
Other_DESA      0.15       1.16       0.14            -0.13             0.43                 0.88                 1.53

                cmp to    z      p   -log2(p)
covariate                                    
Specific_DESA     0.00 8.72 <0.005      58.28
Other_DESA        0.00 1.06   0.29       1.80
---
Concordance = 0.54
Partial AIC = 77024.76
log-likelihood ratio test = 932.44 on 2 df
-log2(p) of ll-ratio test = 672.62

None


In [ ]:
df.Relevant_DESA_Bad.value_counts()

0    4494
1     196
Name: Relevant_DESA_Bad, dtype: int64